In [1]:
%matplotlib inline 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # module for plotting 
import matplotlib.image as mpimg
import sys
import os
import glob
import cPickle as pickle
import re

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [3]:
import shutil
import random
train_data_dir = './data/train'
validation_data_dir = './data/validation'
# Shuffle dataset and split to train set and validation set
for animal in ['dog','cat']:
    if not glob.glob(os.path.join(validation_data_dir,animal,'*')):
        pets = glob.glob(os.path.join(train_data_dir,animal,'*'))
        random.shuffle(pets)
        pet_vals=pets[-2500:]
        for pet in pet_vals:
            shutil.move(pet,os.path.join(validation_data_dir,animal,os.path.basename(pet)))

In [4]:
# Build Convolutional neuro networks and define parameters
img_width, img_height = 128, 128
train_samples = 20000
validation_samples = 4992
epochs = 5
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3,img_width,img_height)
else:
    input_shape = (img_width,img_height,3)
    
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='rmsprop',
             metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 61, 61, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
__________

In [6]:
# Use keras built-in image data generator to read data from hard disk and feed into model batch by batch since the data
# capacity is large to be able to read into memory at once.

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_width,img_height),
                                                   batch_size=batch_size,
                                                   class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                       target_size=(img_width,img_height),
                                                       batch_size=batch_size,
                                                       class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 4992 images belonging to 2 classes.


In [7]:
# Use callbacks to create checkpoint to monitor and save progress. 
from keras import callbacks
fname='weights/conv1.{epoch:02d}-{val_loss:.2f}.h5'
cbks=[callbacks.ModelCheckpoint(filepath=fname,monitor='val_loss',save_best_only=True),
                               callbacks.EarlyStopping(monitor='val_loss',patience=3)]
model.fit_generator(train_generator,steps_per_epoch=train_samples//batch_size,
                   epochs=epochs,validation_data=validation_generator,
                   validation_steps=validation_samples//batch_size,callbacks=cbks)


Epoch 1/5
625/625 [==============================] - 450s 719ms/step - loss: 0.7011 - acc: 0.6603 - val_loss: 0.5407 - val_acc: 0.7210

#### Use Bottleneck features base on VGG16 base model

In [9]:
from keras import applications

In [10]:
top_model_weights_path_vgg = 'bottleneck_Vgg16.h5'
train_samples = 20000
validation_samples = 4992
img_width,img_height = 150,150
epochs=50
batch_size=32

In [13]:
datagen = ImageDataGenerator(rescale=1., featurewise_center=True)

datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

model_VGG16 = applications.VGG16(include_top=False,weights='imagenet')

In [17]:
model_VGG16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [14]:
# Yield bottleneck train data
train_gen = datagen.flow_from_directory(train_data_dir,target_size=(img_width,img_height),
                                                   batch_size=batch_size,
                                                   class_mode=None,shuffle=False)
bn_vgg_train = model_VGG16.predict_generator(train_gen,steps=train_samples//batch_size,verbose=1)
np.save(open('bottleneck_vgg16_train.npy','w'),bn_vgg_train)

# Yield bottleneck validata data
val_gen = datagen.flow_from_directory(validation_data_dir,target_size=(img_width,img_height),
                                                   batch_size=batch_size,
                                                   class_mode=None,shuffle=False)
bn_vgg_val = model_VGG16.predict_generator(val_gen,steps=validation_samples//batch_size,verbose=1)
np.save(open('bottleneck_vgg16_val.npy','w'),bn_vgg_val)

Found 20000 images belonging to 2 classes.
625/625 [==============================] - 365s 584ms/step

In [15]:
bn_vgg_train.shape,bn_vgg_val.shape

((20000L, 4L, 4L, 512L), (4992L, 4L, 4L, 512L))

In [ ]:
# Use VGG16 bottle neck features give a accuracy score of 0.95!

labels_train=[0]*(train_samples/2)+[1]*(train_samples/2)

labels_validation=[0]*(validation_samples/2)+[1]*(validation_samples/2)

top_model = Sequential()

top_model.add(Flatten(input_shape=bn_vgg_train[0].shape))

top_model.add(Dense(256))
top_model.add(Activation('relu'))

top_model.add(Dropout(0.5))

top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

top_model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

top_model.fit(bn_vgg_train,labels_train,epochs=epochs,batch_size=batch_size,
              validation_data=(bn_vgg_val,labels_validation))

top_model.save_weights(top_model_weights_path_vgg)

Train on 20000 samples, validate on 4992 samples
Epoch 1/50
20000/20000 [==============================] - 8s 393us/step - loss: 1.7459 - acc: 0.8848 - val_loss: 1.1544 - val_acc: 0.9251

#### Use Bottleneck features base on Inception V3 base model

In [27]:
from keras.layers import GlobalAveragePooling2D

In [29]:
# Use bottleneck features (InceptionV3)

model_inception_v3=applications.InceptionV3(include_top=False,weights='imagenet')
model_inception_v3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_97[0][0]                  
__________________________________________________________________________________________________
activation_104 (Activation)     (None, None, None, 3 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_98 

In [30]:
top_model_weights_path_inception = 'bottleneck_inception_v3.h5'
train_samples = 20000
validation_samples = 4992
img_width,img_height = 150,150
epochs=50
batch_size=32

In [31]:
# Use Inception V3 image preprocess unit
datagen = ImageDataGenerator(rescale=1.,preprocessing_function=applications.inception_v3.preprocess_input)

# Yield bottleneck train data
train_gen = datagen.flow_from_directory(train_data_dir,target_size=(img_width,img_height),
                                                   batch_size=batch_size,
                                                   class_mode=None,shuffle=False)

bn_inception_train = model_inception_v3.predict_generator(train_gen,steps=train_samples//batch_size,verbose=1)
np.save(open('bottleneck_inception_train.npy','w'),bn_inception_train)

# Yield bottleneck validation data
val_gen = datagen.flow_from_directory(validation_data_dir,target_size=(img_width,img_height),
                                                   batch_size=batch_size,
                                                   class_mode=None,shuffle=False)

bn_inception_val = model_inception_v3.predict_generator(val_gen,steps=validation_samples//batch_size,verbose=1)
np.save(open('bottleneck_inception_val.npy','w'),bn_inception_val)

Found 20000 images belonging to 2 classes.
625/625 [==============================] - 1147s 2s/step

In [32]:
bn_inception_train.shape,bn_inception_val.shape

((20000L, 3L, 3L, 2048L), (4992L, 3L, 3L, 2048L))

In [ ]:
# Train a top FC classification model on the bottleneck data generated from GoogleNet Inception base model

top_model = Sequential()

top_model.add(GlobalAveragePooling2D(input_shape=bn_inception_train.shape[1:]))

top_model.add(Dense(256))
top_model.add(Activation('relu'))

top_model.add(Dropout(0.5))

top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

top_model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

top_model.fit(bn_inception_train,labels_train,epochs=epochs,batch_size=batch_size,
              validation_data=(bn_inception_val,labels_validation))

top_model.save_weights(top_model_weights_path_inception)

Train on 20000 samples, validate on 4992 samples
Epoch 1/50
20000/20000 [==============================] - 7s 336us/step - loss: 0.2502 - acc: 0.9430 - val_loss: 0.1361 - val_acc: 0.9639